In [14]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn.metrics import f1_score

In [65]:
# 生成类别不平衡的数据集
X, y = make_classification(
    n_samples=100, n_features=200, n_informative=2, n_redundant=10, 
    n_clusters_per_class=1, weights=[0.95, 0.05], flip_y=0, random_state=42
)

# 数据标准化
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 数据集划分
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)

# 转换为 PyTorch 张量
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

In [66]:
# 构建普通的简单神经网络（使用交叉熵损失）
class SimpleNN(nn.Module):
    def __init__(self, input_dim):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 2)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [67]:
# 训练模型的函数
def train_model(model, criterion, optimizer, num_epochs=20):
    for epoch in range(num_epochs):
        model.train()
        outputs = model(X_train_tensor)
        loss = criterion(outputs, y_train_tensor)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [68]:
# 使用普通神经网络（交叉熵损失）
model_ce = SimpleNN(X_train.shape[1])
criterion_ce = nn.CrossEntropyLoss()  # 交叉熵损失
optimizer_ce = optim.Adam(model_ce.parameters(), lr=0.001)

# 训练普通神经网络
train_model(model_ce, criterion_ce, optimizer_ce, num_epochs=20)

In [69]:
# 使用 Focal Loss 神经网络
class FocalLoss(nn.Module):
    def __init__(self, alpha=0.25, gamma=2.0):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma

    def forward(self, inputs, targets):
        # 计算预测概率
        log_pt = nn.functional.log_softmax(inputs, dim=1)
        pt = torch.exp(log_pt)
        
        # 计算 Focal Loss
        loss = -self.alpha * (1 - pt) ** self.gamma * log_pt
        return loss.mean()

In [70]:
# 训练 Focal Loss 神经网络的函数
def train_focal_model(model, criterion, optimizer, num_epochs=20):
    for epoch in range(num_epochs):
        model.train()
        outputs = model(X_train_tensor)
        loss = criterion(outputs, y_train_tensor)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [171]:
# 使用 Focal Loss 训练模型
model_focal = SimpleNN(X_train.shape[1])
criterion_focal = FocalLoss(alpha=0.4, gamma=2.0)
optimizer_focal = optim.Adam(model_focal.parameters(), lr=0.001)

# 训练 Focal Loss 神经网络
train_focal_model(model_focal, criterion_focal, optimizer_focal, num_epochs=20)


In [172]:
# 评估模型并计算 F1 分数
def evaluate_model_f1(model, X_test_tensor, y_test_tensor):
    model.eval()
    with torch.no_grad():
        outputs = model(X_test_tensor)
        _, predicted = torch.max(outputs, 1)
        # 计算 F1 分数
        f1 = f1_score(y_test_tensor.numpy(), predicted.numpy())
    return f1

In [173]:
# 评估普通神经网络（交叉熵损失）F1 分数
f1_ce = evaluate_model_f1(model_ce, X_test_tensor, y_test_tensor)
print(f"F1 score of the model with CrossEntropyLoss: {f1_ce:.4f}")

F1 score of the model with CrossEntropyLoss: 0.0000


In [174]:
# 评估 Focal Loss 神经网络的 F1 分数
f1_focal = evaluate_model_f1(model_focal, X_test_tensor, y_test_tensor)
print(f"F1 score of the model with FocalLoss: {f1_focal:.4f}")

F1 score of the model with FocalLoss: 0.4286
